<a href="https://colab.research.google.com/github/veronica1908/MARKETING_E2_A_III/blob/main/c_modelos1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar paquetes y librerías

In [49]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

In [50]:
#Verificar tablas creadas en el script
# Conectarse a la base de datos
conn = sqlite3.connect('db_moviesF')

# Crear un cursor
cur = conn.cursor()

# Obtener una lista de tablas en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])

ratings
movies
usuarios_sel
movies_sel
ratings_final
movies_final
full_ratings


# Sistemas basados en popularidad

##Recomendaciones basado en popularidad

In [51]:
#### Películas mejores calificadas que tengan calificaciÓn
pd.read_sql("""
SELECT movie_title,
       avg(rating) as avg_rat,
       count(*) as read_num
FROM full_ratings
WHERE rating <> 0
GROUP BY movie_title
ORDER BY avg_rat DESC
LIMIT 10
""", conn)

,movie_title,avg_rat,read_num
0,"Shawshank Redemption, The (1994)",4.442922,219
1,"Godfather, The (1972)",4.290850,153
2,Cool Hand Luke (1967),4.290698,43
3,Chinatown (1974),4.255102,49
4,Rear Window (1954),4.253623,69
5,"Princess Bride, The (1987)",4.247863,117
6,Schindler's List (1993),4.246835,158
7,Goodfellas (1990),4.245192,104
8,Dr. Strangelove or: How I Learned to Stop Worr...,4.243902,82
9,American History X (1998),4.237113,97


La película "Shawshank Redemption, The (1994)" tiene el mejor promedio de calificación (4,44) y un total de 219 calificaciones recibidas.

In [52]:
# Películas más vistas con promedio de las calificaciones y cantidad de calificaciones

query = """
SELECT movie_title,
       AVG(CASE WHEN rating = 0 THEN NULL ELSE rating END) AS avg_rat,
       COUNT(*) AS read_num
FROM full_ratings
GROUP BY movie_title
ORDER BY read_num DESC
"""

result = pd.read_sql(query, conn)

# Mostrar las primeras 10 filas del resultado
print(result.head(10))

                                         movie_title   avg_rat  read_num
0                                Forrest Gump (1994)  4.127530       247
1                                Pulp Fiction (1994)  4.189956       229
2                   Shawshank Redemption, The (1994)  4.442922       219
3                                 Matrix, The (1999)  4.123256       215
4                   Silence of the Lambs, The (1991)  4.152913       206
5          Star Wars: Episode IV - A New Hope (1977)  4.178571       196
6                               Jurassic Park (1993)  3.781081       185
7  Star Wars: Episode V - The Empire Strikes Back...  4.183140       172
8                                  Fight Club (1999)  4.225146       171
9                  Terminator 2: Judgment Day (1991)  3.991176       170


Forrest Gump (1994)  es la película más vista, o al menos la más veces calificada, sin embargo su promedio de calificación no estuvo en el top 10 de la consulta anterior, su promedio es de 4.12 con 247 calificaciones.

In [53]:
#### Las películas mejores calificadas por año de lanzamiento ###

#Primero, vamos a extraer el año de lanzamiento, el cual está incluido en el título de película
#Agregar una nueva columna para almacenar el año

conn = sqlite3.connect('db_moviesF')
cursor = conn.cursor()

# Agregar una nueva columna para almacenar el año
cursor.execute("ALTER TABLE full_ratings ADD COLUMN release_year INTEGER")

# Actualizar la nueva columna con el año extraído del título de la película
cursor.execute("""
    UPDATE full_ratings
    SET release_year = CAST(SUBSTR(movie_title, -5, 4) AS INTEGER)
""")

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()


OperationalError: duplicate column name: release_year

In [54]:
# Consulta SQL para seleccionar las primeras filas de la tabla full_ratings
query = "SELECT * FROM full_ratings LIMIT 5"

# Leer los datos en un DataFrame usando pandas
full_ratings_df = pd.read_sql(query, conn)

# Mostrar el DataFrame
print(full_ratings_df)

# Cerrar la conexión
conn.close()

   user_id  movie_id  rating                  movie_title  \
0        1         1     4.0             Toy Story (1995)   
1        1         3     4.0      Grumpier Old Men (1995)   
2        1         6     4.0                  Heat (1995)   
3        1        47     5.0  Seven (a.k.a. Se7en) (1995)   
4        1        50     5.0   Usual Suspects, The (1995)   

                                        genres  release_year  
0  Adventure|Animation|Children|Comedy|Fantasy          1995  
1                               Comedy|Romance          1995  
2                        Action|Crime|Thriller          1995  
3                             Mystery|Thriller          1995  
4                       Crime|Mystery|Thriller          1995  


In [62]:
#Ahora si obtendremos las películas con mejor promedio de calificación y año de publicación
conn = sqlite3.connect('db_moviesF')

# Consulta SQL para obtener las películas mejor calificadas por año de lanzamiento
query = """
SELECT release_year,
       movie_title,
       AVG(rating) AS avg_rating
FROM full_ratings
GROUP BY release_year, movie_title
ORDER BY release_year DESC, avg_rating DESC
"""

# Leer los datos en un DataFrame usando pandas
best_movies_by_year = pd.read_sql(query, conn)

# Mostrar el DataFrame
print(best_movies_by_year)

# Cerrar la conexión
conn.close()

     release_year                             movie_title  avg_rating
0            2016                         Deadpool (2016)    3.710526
1            2014          Guardians of the Galaxy (2014)    4.011364
2            2014                     Interstellar (2014)    3.884615
3            2014        Grand Budapest Hotel, The (2014)    3.744186
4            2013         Wolf of Wall Street, The (2013)    3.825000
..            ...                                     ...         ...
431          1941                     Citizen Kane (1941)    4.035088
432          1940                         Fantasia (1940)    3.769231
433          1940                        Pinocchio (1940)    3.436170
434          1939                Wizard of Oz, The (1939)    3.935897
435          1937  Snow White and the Seven Dwarfs (1937)    3.604839

[436 rows x 3 columns]


Al ver la información por año de lanzamiento, se tiene que la película más recientemente lanzada es Deadpool en 2016, sin embargo, su promedio de calificación no está entre los más altos, siendo de 3.71. Al observar la película con año de lanzamiento más antiguo, se observa que tiene incluso menor promedio de calificación en comparación con la más reciente: Snow White and the Seven Dwarfs (1937) con 3.60.

In [75]:
#Género con mejor promedio de calificación

conn = sqlite3.connect('db_moviesF')

# Ejecutar la consulta SQL
query = """
        SELECT movies_final.genres, AVG(full_ratings.rating) AS avg_rating
        FROM full_ratings
        JOIN movies_final ON full_ratings.movie_id = movies_final.movie_id
        GROUP BY movies_final.genres
        ORDER BY avg_rating DESC;
        """
cursor = conn.cursor()
cursor.execute(query)

result = pd.read_sql(query, conn)

# Mostrar las primeras 10 filas del resultado
print(result.head)

# Cerrar la conexión
conn.close()

<bound method NDFrame.head of                                                 genres  avg_rating
0              Action|Adventure|Comedy|Fantasy|Romance    4.247863
1                                           Comedy|War    4.243902
2            Action|Adventure|Mystery|Romance|Thriller    4.208333
3                               Crime|Mystery|Thriller    4.194340
4                             Action|Adventure|Western    4.180000
..                                                 ...         ...
196  Adventure|Animation|Children|Comedy|Fantasy|Sc...    2.737500
197                       Action|Comedy|Crime|Thriller    2.726190
198                      Comedy|Fantasy|Romance|Sci-Fi    2.704918
199                                Action|Crime|Sci-Fi    2.568627
200                       Action|Comedy|Sci-Fi|Western    2.166667

[201 rows x 2 columns]>


El género de película con mejor promedio de calificación es Action|Adventure|Comedy|Fantasy|Romance con 4.24, mientras que el de peor promedio de calificación es Action|Comedy|Sci-Fi|Western con 2,13.

## Sistema de recomendación basado en contenido un solo producto - Manual

In [63]:
conn = sqlite3.connect('db_moviesF')

# Consulta SQL para obtener todos los registros de la tabla 'full_ratings'
query = 'SELECT * FROM full_ratings'

# Leer los datos en un DataFrame usando pandas
movies = pd.read_sql(query, conn)

# Mostrar información sobre el DataFrame antes de convertir 'release_year' a entero
movies.info()

# Convertir la columna 'release_year' a tipo entero
movies['release_year'] = movies['release_year'].astype('int')

# Mostrar información sobre el DataFrame después de la conversión
movies.info()

# Cerrar la conexión
conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32507 entries, 0 to 32506
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       32507 non-null  int64  
 1   movie_id      32507 non-null  int64  
 2   rating        32507 non-null  float64
 3   movie_title   32507 non-null  object 
 4   genres        32507 non-null  object 
 5   release_year  32507 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32507 entries, 0 to 32506
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       32507 non-null  int64  
 1   movie_id      32507 non-null  int64  
 2   rating        32507 non-null  float64
 3   movie_title   32507 non-null  object 
 4   genres        32507 non-null  object 
 5   release_year  32507 non-null  int64  
dtypes: float64(1), int64(3), obje

In [65]:
# escalar para que año esté en el mismo rango
sc=MinMaxScaler()
movies[["year_sc"]]=sc.fit_transform(movies[['release_year']])

In [68]:
#### convertir a dummies
